<a href="https://colab.research.google.com/github/kurtlee1984/Modification_code/blob/main/YOLOv4_Tensorflow_modifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# 將 Colab 與 google_drive 綁定
# 連接到指定帳號,並複製授權碼貼上

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 檢視現在設備Cuda的版本

!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [4]:
# 當前設備顯卡參數列表明細

!nvidia-smi

Wed Jul 21 10:33:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# 下載資料集,並於 google_drive 取得壓縮檔的id
# id取得方法,到 google_drive 指定壓縮檔,滑鼠右鍵選擇取得連結,更改權限由限制->知道連結的使用者,複製連結在筆記本上
# https://drive.google.com/file/d/這裡就是壓縮檔的id/view?usp=sharing

# 原始檔名
# %cd /content
# !gdown --id '1qE85RXsrgCksM9lo7TnToQNLeyx-YUse' -O dataset.zip

# 檔名改版(原設)
%cd /content
!gdown --id '1XoCtwKhE_27vk-OJ7WfpAp2Z0M6NAXHc' -O dataset.zip 

# 原始檔名(老師)
# %cd /content
# !gdown --id '1eklaKQIfh6EUtYWBfW3MXeBZOxXFlTZb' -O dataset.zip

# 檔名改版(老師)
# %cd /content
# !gdown --id 1dJGzRNjloO_4VFsUi3mcBsIovAWxmvOf -O dataset.zip

/content
Downloading...
From: https://drive.google.com/uc?id=1XoCtwKhE_27vk-OJ7WfpAp2Z0M6NAXHc
To: /content/dataset.zip
270MB [00:09, 29.6MB/s]


In [6]:
# 解壓縮,注意壓縮檔名稱需與代碼一致
# 解壓縮完,於右邊選擇檔案,看有無已解壓縮的資料夾

!unzip -q dataset.zip; rm dataset.zip

In [ ]:
# 從Github下載yolo-v4-tf.keras資料庫

# !git clone https://github.com/taipingeric/yolo-v4-tf.keras.git

In [7]:
# 將 google_drve 的 yolo-v4-tf.keras資料夾綁定成系統路徑

import sys
FOLDER_PATH = '/content/drive/MyDrive/yolo-v4-tf.keras' 
sys.path.append(FOLDER_PATH)

In [8]:
#  導入yolo-v4-tf.keras的函示庫與必要套件

from utils import DataGenerator, read_annotation_lines
from models import Yolov4
from config import yolo_config
import os

In [ ]:
# 定義train與vail的標註檔和圖像檔的位置路徑, (因為有進行data augmentation,故已將train和val分開), data augmentatio後檔案已變形不適合val

# 若未進行增強或是增強是在 DataGenerator 時進行,可以使用 train_test_split 區分 test 和 valid, 但 dataset 也要相對應的做改變
# rain_lines, val_lines = read_annotation_lines('./content/train_txt/annotations.txt', test_size=0.1)

train_lines = read_annotation_lines('./train_txt/annotations.txt')
TRAIN_FOLDER_PATH  = './train_img'

valid_lines = read_annotation_lines('./valid_txt/annotations.txt')
VALID_FOLDER_PATH = './valid_img'

# 定義類別檔的位置路徑

class_name_path = os.path.join(FOLDER_PATH, 'class_names/food_classes.txt')

# 設定 train 和 valid 的 dataGenerator

data_gen_train = DataGenerator(train_lines, class_name_path, TRAIN_FOLDER_PATH)
data_gen_valid = DataGenerator(valid_lines, class_name_path, VALID_FOLDER_PATH)

# 導入模型函數與架構,不導入權重參數

model = Yolov4(weight_path=None, 
               class_name_path=class_name_path)

# 餵入訓練資料,設定epochs數,這裡設定為100

model.fit(data_gen_train, 
          initial_epoch=0,
          epochs=500, 
          val_data_gen=data_gen_valid)

In [ ]:
# 設定標註和圖像的路徑, 並使用train_test_split 區分 train 和 valid, test_size(可調整)

# train_lines, val_lines = read_annotation_lines('./dataset/train_txt/anno.txt', test_size=0.1)
# IMG_FOLDER_PATH = './dataset/train_img'
# class_name_path = os.path.join(FOLDER_PATH, 'class_names/food_classes.txt')
# data_gen_train = DataGenerator(train_lines, class_name_path, IMG_FOLDER_PATH)
# data_gen_val = DataGenerator(val_lines, class_name_path, IMG_FOLDER_PATH)

# model = Yolov4(weight_path=None, 
#                class_name_path=class_name_path)

# model.fit(data_gen_train, 
#           initial_epoch=0,
#           epochs=500, 
#           val_data_gen=data_gen_val)

In [ ]:
# 將模型儲存到 google_drive,  和 colab 上

model.yolo_model.save('./content/drive/MyDrive/model.h5')
model.yolo_model.save(('./content/model.h5')

In [ ]:
# 將權重儲存到 google_drive, 和 colab 上

model.yolo_model.save_weights('./content/drive/MyDrive/yolov4_weights.h5')
model.yolo_model.save_weights('./content/yolov4_weights.h5')

In [ ]:
# 導入 inference 所需要的套件與函式庫

import cv2
import numpy as np
import os
from models import Yolov4
from glob import glob

In [ ]:
# 下載已訓練好的權重,並將檔名成 model.h5

!gdown --id "" -O model.h5

In [ ]:
# 導入模型和權重檔案,建立模型
# 設定類別路徑

model = Yolov4(class_name_path = os.path.join(FOLDER_PATH, 'class_names', 'food_classes.txt'))  

model.load_model('model.h5')

In [ ]:
# 設定test圖片路徑

img_paths = glob('/test_img/*.jpg')
len(img_paths)

In [ ]:
# 隨機選擇 test 圖片進行物件預測

path = np.random.choice(img_paths, 1)[0]
model.predict(path)